In [1]:
import tensorflow as tf
from tensorflow.python.ops.numpy_ops import np_config

np_config.enable_numpy_behavior()

raw_image_dataset = tf.data.TFRecordDataset('../images.tfrecords')

# Create a dictionary describing the features.
image_feature_description = {
    'label': tf.io.FixedLenFeature([], tf.int64),
    'image_raw': tf.io.FixedLenFeature([], tf.string),
}

def _parse_image_function(example_proto):
  example = tf.io.parse_single_example(example_proto, image_feature_description)
  label, image = example["label"], example["image_raw"]
  label = tf.one_hot(label, depth=10)
  # print(tf.io.parse_single_example(example_proto, image_feature_description))
  # image_raw = image_features['image_raw']
  image = tf.io.decode_raw(image, tf.uint8)
  image = tf.reshape(image, (640, 240, 1))
  image = image.astype("float32") / 255
  
  return image, label
  
  # return tf.io.parse_single_example(example_proto, image_feature_description)

parsed_image_dataset = raw_image_dataset.map(_parse_image_function)
parsed_image_dataset

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


<MapDataset shapes: ((640, 240, 1), (10,)), types: (tf.float32, tf.float32)>

In [ ]:
for image_features in parsed_image_dataset:
  image = image_features[0]
  label = image_features[1]
  print(label)
  print(image)
  break

In [2]:
dataset = parsed_image_dataset.shuffle(1000).batch(64)

In [ ]:
for batch in dataset:
  print(batch)
  break

In [3]:
from tensorflow.keras import models
from tensorflow.keras import layers

model = models.Sequential()
layer1 = layers.Conv2D(filters=32, input_shape=(640, 240, 1),
  kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu')
model.add(layer1)
model.add(layers.MaxPool2D(pool_size=(2,2)))

layer2 = layers.Dense(128, activation='relu')
model.add(layer2)

# model.add(layers.Dropout(0.25))
model.add(layers.Flatten())

layer3 = layers.Dense(10, activation='softmax')
model.add(layer3)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 640, 240, 32)      320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 320, 120, 32)     0         
 )                                                               
                                                                 
 dense (Dense)               (None, 320, 120, 128)     4224      
                                                                 
 flatten (Flatten)           (None, 4915200)           0         
                                                                 
 dense_1 (Dense)             (None, 10)                49152010  
                                                                 
Total params: 49,156,554
Trainable params: 49,156,554
Non-trainable params: 0
____________________________________________

In [4]:

# myadam = tf.keras.optimizers.Adam(
#     learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False, name='Adam'
# )

model.compile(optimizer='adam',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

# 降低学习率
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='loss',verbose=1, factor=0.2,patience=1, min_lr=0.0001)

# 提前终止
# es=tf.keras.callbacks.EarlyStopping(monitor='loss',verbose=1,mode='min',patience=2)
 
# model.fit(dataset, epochs=10, callbacks=[reduce_lr ,es])
model.fit(dataset, epochs=20)

Epoch 1/20
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
313/313 [==============================] - 2392s 8s/step - loss: 285.3352 - accuracy: 0.4642
Epoch 2/20
313/313 [==============================] - 2395s 8s/step - loss: 8411.6387 - accuracy: 0.5377
Epoch 3/20
313/313 [==============================] - 2298s 7s/step - loss: 14490.3301 - accuracy: 0.6810
Epoch 4/20
313/313 [==============================] - 2639s 8s/

KeyboardInterrupt: 